In [3]:
!pip install dash


In [5]:
!pip install jupyter-dash

In [8]:
!pip install dash-leaflet


In [9]:

import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

import dash_leaflet as dl


# Load Data
df = px.data.tips()


# Build App
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("JupyterDash Demo"),
    dcc.Graph(id='graph'),
    html.Label([
        "colorscale",
        dcc.Dropdown(
            id='colorscale-dropdown', clearable=False,
            value='plasma', options=[
                {'label': c, 'value': c}
                for c in px.colors.named_colorscales()
            ])
    ]),
    dl.Map(dl.TileLayer(), style={'width': '1000px', 'height': '500px'}),
])

# Define callback to update graph
@app.callback(
    Output('graph', 'figure'),
    [Input("colorscale-dropdown", "value")]
)
def update_figure(colorscale):
    return px.scatter(
        df, x="total_bill", y="tip", color="size",
        color_continuous_scale=colorscale,
        render_mode="webgl", title="Tips"
    )
# Run app and display result inline in the notebook
app.run_server(mode='inline')
#app.run_server(mode='inline',host="0.0.0.0",port=1005)


In [1]:
!pip install gpxpy

  Created wheel for gpxpy: filename=gpxpy-1.5.0-py3-none-any.whl size=42899 sha256=325dd7402a6fcceb6e976d0fba16b5015a36cfbc2a51916accfe0f2e6ce0ed87
  Stored in directory: c:\users\jesus alcocer\appdata\local\pip\cache\wheels\7e\9b\8d\b4812540cd01add3ca698dc5903c53b99d15ffbd61f23fdf0a
Successfully built gpxpy


In [2]:
import gpxpy
import gpxpy.gpx

import pandas as pd

In [3]:
with open('HU-GR.gpx', 'r') as gpx_file:
    gpx = gpxpy.parse(gpx_file)

route_info = []

for track in gpx.tracks:
    for segment in track.segments:
        for point in segment.points:
            route_info.append({
                'latitude': point.latitude,
                'longitude': point.longitude,
                'elevation': point.elevation
            })

route_df = pd.DataFrame(route_info)
route_df.head()                

,latitude,longitude,elevation
0,47.634836,19.079729,101.675182
1,47.634213,19.080162,101.675182
2,47.632601,19.082329,101.428709
3,47.632461,19.082461,101.455441
4,47.632335,19.082412,101.476433


In [4]:
!pip install folium


In [25]:
import folium


route_map = folium.Map(
    location=[48.4, 19.3],
    zoom_start=5,
    tiles='OpenStreetMap',
    width=1024,
    height=600
)

coordinates = [tuple(x) for x in route_df[['latitude', 'longitude']].to_numpy()]
folium.PolyLine(coordinates, weight=6).add_to(route_map)

display(route_map)

In [10]:

import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

import dash_leaflet as dl


# Load Data
df = px.data.tips()


# Build App
app = JupyterDash(__name__)
# app.layout = html.Div([
#     html.H1("JupyterDash MAP"),
#     dl.Map(dl.TileLayer(), style={'width': '1000px', 'height': '500px'}),
#     dl.Map(dl.TileLayer(), style={'width': '1000px', 'height': '500px'}),
# ])
app.layout = html.Div(
    dl.Map(
        [
            dl.LayersControl(
                [
                    dl.BaseLayer(
                        dl.TileLayer(),
                        name="OpenStreetMaps",
                        checked=True,
                    ),
                    dl.BaseLayer(
                        dl.TileLayer(
                            url="https://www.ign.es/wmts/mapa-raster?request=getTile&layer=MTN&TileMatrixSet=GoogleMapsCompatible&TileMatrix={z}&TileCol={x}&TileRow={y}&format=image/jpeg",
                            attribution="IGN",
                        ),
                        name="IGN",
                        checked=False,
                    ),
                ],
            ),
          
        ],
        zoom=7,
        center=(40.0884, -3.68042),
    ),
    style={
        "height": "100vh",
    },
)



# Define callback to update graph
@app.callback(
    Output('graph', 'figure'),
    [Input("colorscale-dropdown", "value")]
)
def update_figure(colorscale):
    return px.scatter(
        df, x="total_bill", y="tip", color="size",
        color_continuous_scale=colorscale,
        render_mode="webgl", title="Tips"
    )
# Run app and display result inline in the notebook
app.run_server(mode='inline')
#app.run_server(mode='inline',host="0.0.0.0",port=1005)


In [24]:
import dash
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output


import dash_leaflet as dl
import gpxpy




icon = {
    "iconUrl": "https://leafletjs.com/examples/custom-icons/leaf-green.png",
    "shadowUrl": "https://leafletjs.com/examples/custom-icons/leaf-shadow.png",
    "iconSize": [38, 95],  # size of the icon
    "shadowSize": [50, 64],  # size of the shadow
    "iconAnchor": [
        22,
        94,
    ],  # point of the icon which will correspond to marker's location
    "shadowAnchor": [4, 62],  # the same for the shadow
    "popupAnchor": [
        -3,
        -76,
    ],  # point from which the popup should open relative to the iconAnchor
}


def get_data():
    gpx_file = open("HU-GR.gpx", "r")
    gpx = gpxpy.parse(gpx_file)
    markers = []
    for waypoint in gpx.waypoints:
        markers.append(
            dl.Polyline (
            #dl.Marker(
                title=waypoint.name,
                position=(waypoint.latitude, waypoint.longitude),
                icon=icon,
                children=[
                    dl.Tooltip(waypoint.name),
                    dl.Popup(waypoint.name),
                ],
            )
        )
    cluster = dl.MarkerClusterGroup(id="markers", children=markers)
    return cluster


app = JupyterDash(__name__)
#app = dash.Dash(__name__)
app.layout = html.Div(
    
    dl.Map(
        [
            dl.TileLayer(),
            get_data(), #
        ],
        zoom=6,
        center=(48.4, 19.3),
    ), #end dash leaflet map

    style={
        "height": "100vh",
    },
)

app.run_server(mode='inline')
#app.run_server(mode='inline',host="0.0.0.0",port=1005)


In [33]:
import dash
import dash_leaflet as dl
import dash_html_components as html

app = JupyterDash(__name__)
#app = dash.Dash()


app.layout = html.Div([
    dl.Map(style={'width': '1000px', 'height': '500px'}, center=[48.4, 19.3], zoom=5, children=dl.TileLayer())
])

if __name__ == '__main__':
    #app.run_server(debug=False)
    app.run_server(mode='inline')

In [44]:
!pip install gpx_converter

In [45]:
from gpx_converter import Converter

In [42]:
import json

In [46]:
#dic = Converter(input_file='HU-GR.gpx').gpx_to_dictionary(latitude_key='latitude', longitude_key='longitude')

In [50]:
!pip install geojson

In [59]:
import geojson
with open('./HU-GR_geojson/tracks.geojson') as f:
    gj = geojson.load(f)
    

In [1]:
import dash
import dash_leaflet as dl
import dash_html_components as html

app = JupyterDash(__name__)
#app = dash.Dash()

layer= dl.GeoJSON(data=gj)
#layer = dl.Polyline(position=(waypoint.latitude, waypoint.longitude)coordinates, weight=6)
           
app.layout = html.Div([
    dl.Map([layer,dl.TileLayer()],style={'width': '1000px', 'height': '500px'}, center=[48.4, 19.3], zoom=5, ),
    dl.Map([layer,dl.TileLayer()],style={'width': '1000px', 'height': '500px'}, center=[48.4, 19.3], zoom=5, )
])

if __name__ == '__main__':
    #app.run_server(debug=False)
    app.run_server(mode='inline')

C:\Users\JESUSA~1\AppData\Local\Temp/ipykernel_11768/1027316431.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


NameError: name 'JupyterDash' is not defined